In [21]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
os.chdir("/mnt/lareaulab/reliscu/projects/Chronocell/data/protein_degradation_rates")

In [ ]:
file_list = glob.glob("*standardized.csv")
subset_columns = ['Gene', 'Half-life']

df_list = []
for file in [file for file in file_list if "Human" in file]:
    df = pd.read_csv(file)
    study = df['Study'].iloc[0]
    cell_type = df['Cell_type'].iloc[0]
    df = df[subset_columns]
    df = df.rename(columns={"Half-life": f"Half-life_{study}", 
                            "Cell_type": f"Cell_type_{study}"}) 
    df_list.append(df)

In [19]:
df_list[1].head()

,Study,Species,Cell_type,Method,Gene,Half-life_Mathieson2018
0,Mathieson2018,Human,B cells,Dynamic SILAC,AAAS,81.8375
1,Mathieson2018,Human,B cells,Dynamic SILAC,AACS,30.5240
2,Mathieson2018,Human,B cells,Dynamic SILAC,AAMP,21.2640
3,Mathieson2018,Human,B cells,Dynamic SILAC,AAR2,35.4590
4,Mathieson2018,Human,B cells,Dynamic SILAC,AARS,39.9375


In [28]:
subset_columns = ['Gene', 'Half-life']
df = pd.merge(df_list[0], df_list[1], on="Gene", how="outer")

In [29]:
df.head()

,Gene,Half-life_Boisvert2011,Half-life_Mathieson2018
0,EXOC4,999.000000,67.0455
1,FAM82A2,999.000000,NaN
2,NDUFAF4,314.628167,74.3840
3,NaN,999.000000,NaN
4,NaN,999.000000,NaN


In [30]:
df[pd.isna(df['Gene'])]

,Gene,Half-life_Boisvert2011,Half-life_Mathieson2018
3,NaN,999.000000,NaN
4,NaN,999.000000,NaN
5,NaN,1001.549653,NaN
6,NaN,999.000000,NaN
7,NaN,999.000000,NaN
...,...,...,...
413,NaN,41.374595,NaN
414,NaN,999.000000,NaN
415,NaN,653.973750,NaN
416,NaN,46.191373,NaN
